In [7]:
from utils.utils import get_df_serie_temporal, agregar_columna_mes, categorize_precipitation, reshample_time_serie, describe_historical_serie, autocorrelacion
from graphics_utils.express import fig_boxplot, fig_pie
from statistics_utils.eda.timeseries import decompose_time_series, correlation
#from figuras import combined_figures
from api_somo import fetch_qf
#from earth_engine import ee_precipitation_data
from graphics_utils.go import multi_scatter, multi_boxplot, fig_pie_chart_by_month
import kaleido
import numpy as np

In [8]:
# PARAMETROS PARA OBTENER SERIE DE TIEMPO DESDE LAS APIS
api_filtrate_seire = "http://127.0.0.1:8000/time-series/qfcontroler/filtrate_time_serie/"
params = {
    "station_name": "tumaco",
    "variable_name": "Precipitación acumulada",
    "start_date": "2022-01-01",
    "end_date": "2022-12-31"
}

collection = {
    "collection": "UCSB-CHG/CHIRPS/DAILY"  # Cambia esto por tu colección específica
}

In [10]:
# OBTENCION DE SERIES 

# serie insitu en df
insitu = fetch_and_process_data(api_filtrate_seire, params)
pie_fig = fig_pie_chart_by_month(insitu)
pie_fig.show()

Error al conectar con la API: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /time-series/qfcontroler/filtrate_time_serie/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001B866880DA0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))


AttributeError: 'NoneType' object has no attribute 'index'

In [ ]:
# PRE-PROCESAMIENTO DE LAS SERIES
insitu.loc[insitu['qf'] != 1, 'value'] = np.nan
insitu = categorize_precipitation(insitu, params)
insitu = reshample_time_serie(insitu, 'D', 'sum')
#insitu = categorize_precipitation(insitu, params)


In [27]:
########### PROCESAMIENTO DE LOS DATOS ###########
# METODOS DE IMPUTACION DE DATOS
#imputado = imputar_knn(insitu, n_neighbors=60)

# DESCOMPOSICION DE LA SERIE EN PATRONES ESTACIONALES
IS = decompose_time_series(insitu)
#print(IS)

#autocorrelaciones de las series
#insitu_acorr = autocorrelacion(insitu, params)
#ee_acorr = autocorrelacion(ee_serie, params)

In [28]:
#### GRAFICADO###

scatter = multi_scatter( Tendencia=IS['trend'], Estacional=IS['seasonal'], Residuo=IS['resid'])
scatter.show()


In [29]:
corr_trend = IS['trend'].iloc[:, 0].corr(insitu.iloc[:, 0])  # Asegúrate de que solo seleccionas una columna
corr_seasonal = IS['seasonal'].iloc[:, 0].corr(insitu.iloc[:, 0])  # Asegúrate de que solo seleccionas una columna
corr_resid = IS['resid'].iloc[:, 0].corr(insitu.iloc[:, 0])  # Asegúrate de que solo seleccionas una columna

print('Tendencia:', round(corr_trend, 3))
print('Estacionalidad:', round(corr_seasonal, 3))
print('Residuo:', round(corr_resid, 3) )


Tendencia: 0.406
Estacionalidad: 0.422
Residuo: 0.766
